# Gaussian Process Latent Variable Models with SVI 

## Introduction 

In this notebook we demonstrate the GPLVM model class introduced in [Lawrence, 2005](https://proceedings.neurips.cc/paper/2003/file/9657c1fffd38824e5ab0472e022e577e-Paper.pdf) and its Bayesian incarnation introduced in [Titsias, 2010](http://proceedings.mlr.press/v9/titsias10a/titsias10a.pdf).

GPLVMs use Gaussian processes in an unsupervised context, where a low dimensional representation of the data ($X \equiv \{\mathbf{x}_{n}\}_{n=1}^{N}\in \mathbb{R}^{N \times Q}$) is learnt given some high dimensional real valued observations $Y \equiv \{\mathbf{y}_{n}\}_{n=1}^{N} \in \mathbb{R}^{N \times D}$. $Q < D$ provides dimensionality reduction. The forward mapping ($X \longrightarrow Y$) is governed by GPs independently defined across dimensions $D$. Q (the dimensionality of the latent space is usually fixed before hand).

One can either learn point estimates for each $\mathbf{x}_{n}$ by maximizing the GP marginal likelihood (use `gpytorch.mlls.ExactMarginalLogLikelihood` for this) jointly wrt. the kernel hyperparameters $\theta$ and the latent inputs $\mathbf{x}_{n}$. Alternatively, one can variationally integrate out $X$ by using the sparse variational formulation where a variational distribution $q(X) = \prod_{n=1}^{N}\mathcal{N}(\mathbf{x}_{n}; \mu_{n}, s_{n}\mathbb{I}_{Q})$.This tutorial focuses on the latter. 

The probabilistic model is: 

\begin{align*}
\textrm{ Prior on latents: } p(X) &= \displaystyle \prod _{n=1}^N \mathcal{N} (\mathbf{x}_{n};\mathbf{0}, \mathbb{I}_{Q}),\\
\textrm{Prior on mapping: }    p(\mathbf{f}|X, \mathbf{\theta}) &= \displaystyle \prod_{d=1}^{D}\mathcal{N}(\mathbf{f}_{d}; \mathbf{0}, K_{ff}^{(d)}),\\
\textrm{Data likelihood: }  p(Y| \mathbf{f}, X) &= \prod_{n=1}^N \prod_{d=1}^D \mathcal{N}(y_{n,d}; \mathbf{f}_{d}(\mathbf{x}_{n}), \sigma^{2}_{y}),
\end{align*}


In [5]:
# Standard imports
import matplotlib.pylab as plt
import torch 
import numpy as np
import sklearn.datasets as skd

# gpytorch imports
from gpytorch.mlls import VariationalELBO
from gpytorch.priors import NormalPrior

%matplotlib inline
%load_ext autoreload
%autoreload 2

### Set up training data 

We generate a high dimensional Y dataset by using some generic transformations of the latent X (not used in training). The ground truth latent X has structure (3 separated clusters with different dispersion) and it would be interesting to see if the learnt X post-training recovers similar structure to the ground truth. (Although, it will never be able to recover the exact ground truth latent X due to non-identifiabilities.)

In [ ]:
n_samples = 500

def err(): return np.random.normal(size=n_samples)*0.05

X, labels = skd.make_blobs(n_samples=n_samples, random_state=42,
                              cluster_std=[1.0, 2.5, 0.5])
                              
Y = float_tensor(np.vstack([
            0.1 * (X[:, 0] + X[:, 1])**2 - 3.5 + err(),
            0.01 * (X[:, 0] + X[:, 1])**3 + err(),
            2 * np.sin(0.5*(X[:, 0] + X[:, 1])) + err(),
            2 * np.cos(0.5*(X[:, 0] + X[:, 1])) + err(),
            4 - 0.1*(X[:, 0] + X[:, 1])**2 + err(),
            1 - 0.01*(X[:, 0] + X[:, 1])**3 + err(),
        ]).T)

### Setting up the model

We will be using the GPLVM model 

Since we're performing VI, we'll be using a `~gpytorch.models.ApproximateGP`. Similar to the [SVGP example](./SVGP_Regression_CUDA.ipynb), we'll use a `VariationalStrategy` and a `CholeskyVariationalDistribution` to define the posterior approximation 

